# Remittance to the Philippines – Statistical Analysis

**Dataset Source:**  
https://www.kaggle.com/datasets/joshbuttler/remittance-to-the-philippines

**Input File:**  
data/processed/remittance_cleaned.csv

**Purpose:**  
Apply formal statistical methods to:
- Quantify relationships between variables
- Test differences across groups
- Assess statistical significance of observed patterns

In [ ]:
import pandas as pd
import numpy as np

from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", "{:,.2f}".format)

In [ ]:
DATA_PATH = "../data/processed/remittance_cleaned.csv"
df = pd.read_csv(DATA_PATH)

df.head()

In [ ]:
# Identify numeric target variable
amount_col = "amount" if "amount" in df.columns else df.select_dtypes(np.number).columns[0]

# Identify grouping variables
country_cols = [c for c in df.columns if "country" in c.lower() or "origin" in c.lower()]
channel_cols = [c for c in df.columns if "channel" in c.lower() or "method" in c.lower()]

amount_col, country_cols, channel_cols

In [ ]:
numeric_df = df.select_dtypes(np.number)

pearson_corr = numeric_df.corr(method="pearson")
spearman_corr = numeric_df.corr(method="spearman")

pearson_corr

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(pearson_corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Pearson Correlation Matrix")
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(spearman_corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Spearman Correlation Matrix")
plt.show()

In [ ]:
sample = df[amount_col].dropna().sample(min(500, len(df)), random_state=42)

shapiro_test = stats.shapiro(sample)
shapiro_test

In [ ]:
if country_cols:
    country_col = country_cols[0]
    top_countries = (
        df[country_col]
        .value_counts()
        .head(2)
        .index
        .tolist()
    )

    group1 = df[df[country_col] == top_countries[0]][amount_col]
    group2 = df[df[country_col] == top_countries[1]][amount_col]

    ttest_result = stats.ttest_ind(group1, group2, equal_var=False)
    mw_test = stats.mannwhitneyu(group1, group2, alternative="two-sided")

    ttest_result, mw_test

In [ ]:
if country_cols:
    top5 = (
        df[country_col]
        .value_counts()
        .head(5)
        .index
        .tolist()
    )

    samples = [
        df[df[country_col] == c][amount_col]
        for c in top5
    ]

    anova_result = stats.f_oneway(*samples)
    kruskal_result = stats.kruskal(*samples)

    anova_result, kruskal_result

In [ ]:
def cohens_d(x, y):
    nx, ny = len(x), len(y)
    pooled_std = np.sqrt(
        ((nx - 1)*x.std()**2 + (ny - 1)*y.std()**2) / (nx + ny - 2)
    )
    return (x.mean() - y.mean()) / pooled_std

if country_cols:
    cohens_d(group1, group2)

In [ ]:
if channel_cols:
    channel_col = channel_cols[0]
    channel_groups = [
        df[df[channel_col] == c][amount_col]
        for c in df[channel_col].unique()
    ]

    stats.kruskal(*channel_groups)

In [ ]:
mean = df[amount_col].mean()
sem = stats.sem(df[amount_col])
ci = stats.t.interval(
    confidence=0.95,
    df=len(df[amount_col]) - 1,
    loc=mean,
    scale=sem
)

mean, ci

In [ ]:
summary_stats = df.groupby(country_col)[amount_col].agg(
    mean="mean",
    median="median",
    std="std",
    count="count"
).reset_index()

summary_stats.head()

## Key Statistical Findings

- Remittance amounts are generally non-normally distributed.
- Statistically significant differences exist between major sending countries.
- Multi-group tests indicate structural variation across remittance sources.
- Effect size analysis suggests that differences are economically meaningful, not just statistically significant.
- Channel-based differences (if applicable) may reflect access and cost factors.